In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn import linear_model as lm
from sklearn.feature_selection import r_regression
from sklearn.metrics import r2_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
approval = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-FKWVTTFtJT6u56e0bqdfoMcXvDO1DUChsJ3jQAMB2lZk2SMqVfmg7dGjclTYkYWz-Pm5lfcLPjp4/pub?output=csv')
# approval = approval.reset_index()
# approval = approval.rename({'index':'poll_id'}, axis=1)
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,tracking,approve,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp
0,Donald Trump (2nd term),All polls,TIPP Insights,League of American Workers,11/25/2025,11/29/2025,1483.0,A,1.568241,1.299610,NaN,43.0,47.0,-4.0,43.84750,49.67333,-5.825829,https://issuesinsights.com/2025/12/01/after-br...,11/29/2025,15:00:36 2 Dec 2025
1,Donald Trump (2nd term),Adults,TIPP Insights,League of American Workers,11/25/2025,11/29/2025,1483.0,A,1.568241,1.299610,NaN,43.0,47.0,-4.0,42.95068,50.33221,-7.381531,https://issuesinsights.com/2025/12/01/after-br...,11/29/2025,15:00:45 2 Dec 2025
2,Donald Trump (2nd term),Cost,YouGov,Economist,11/28/2025,12/1/2025,1628.0,A,0.950390,0.950390,NaN,32.0,59.0,-27.0,32.22042,59.51837,-27.297950,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/1/2025,15:00:58 2 Dec 2025
3,Donald Trump (2nd term),Voters,YouGov,Economist,11/28/2025,12/1/2025,1456.0,RV,0.844276,0.844276,NaN,42.0,56.0,-14.0,42.61599,54.94342,-12.327430,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/1/2025,15:00:42 2 Dec 2025
4,Donald Trump (2nd term),Adults,YouGov,Economist,11/28/2025,12/1/2025,1628.0,A,0.773958,0.773958,NaN,39.0,58.0,-19.0,39.29380,57.84279,-18.548990,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/1/2025,15:00:45 2 Dec 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,Donald Trump (2nd term),Immigration,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,46.0,39.0,7.0,47.87135,39.35921,8.512138,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,15:00:56 2 Dec 2025
3259,Donald Trump (2nd term),Economy,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,42.0,36.0,6.0,44.91630,36.05471,8.861591,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,15:00:54 2 Dec 2025
3260,Donald Trump (2nd term),Adults,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,47.0,41.0,6.0,47.82215,40.12941,7.692741,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,15:00:45 2 Dec 2025
3261,Donald Trump (2nd term),All polls,InsiderAdvantage,NaN,1/20/2025,1/20/2025,800.0,RV,1.093180,0.000000,NaN,56.0,39.0,17.0,52.45134,42.13375,10.317590,https://insideradvantage.com/first-insideradva...,1/20/2025,15:00:36 2 Dec 2025


In [4]:
# approval[approval['pollster']=='YouGov'][approval['subgroup']=='All polls']

In [5]:
approval = approval[~approval['subgroup'].isin(['Weak', 'Strong', 'Economy', 'Cost', 'Immigration', 'Trade', 'Adults', 'All polls'])]

In [6]:
approval.to_csv('trump_approval_polls_RV.csv')

In [7]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'enddate', 'samplesize', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp'], dtype=object)

In [8]:
approval.shape

(574, 20)

In [9]:
# dates = approval['Dates'].str.extract(r'(\d{1,2}\/\d{1,2}) \- (\d{1,2}\/\d{1,2}), (\d{4})')
# dates

In [10]:
# dates['end_date'] = dates[1] + '/' + dates[2]
# dates['end_date'] = pd.to_datetime(dates['end_date'])
# dates

In [11]:
# approval['Dates'] = dates['end_date']
# approval.head()

In [12]:
# samples = approval['Sample'].str.extract(r'[0-9,]{3,5} (\w{1,2})@@(\d{3,4})')
# samples

In [13]:
# approval['population'] = samples[0]
# approval['sample_size'] = samples[1].astype(int)
# approval.head()

In [14]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [15]:
ratings['Pollster'].values

array(['AtlasIntel', 'Marquette University Law School',
       'The Washington Post', 'Research & Polling Inc.',
       'The New York Times/Siena College', 'Cygnal', 'SurveyUSA',
       'Siena College', 'Beacon Research/Shaw & Co. Research',
       'Alaska Survey Research', 'OnMessage Inc.', 'TIPP Insights',
       'Emerson College', 'Landmark Communications',
       'Monmouth University', 'Data Orbital',
       'University of North Florida', 'MassINC Polling Group',
       'Marist College', 'Public Policy Institute of California',
       'Public Policy Polling', 'University of California Berkeley',
       'CBS News/The New York Times', 'Christopher Newport University',
       'Mason-Dixon Polling & Strategy', 'Strategic Vision LLC',
       'Suffolk University', 'Stockton University',
       'Field Research Corp. (Field Poll)', 'Quinnipiac University',
       'Los Angeles Times',
       'Hart Research Associates/Public Opinion Strategies',
       'Public Opinion Strategies', 'Susquehan

In [16]:
ratings[ratings['Pollster'].str.contains('North Star')]#.sum()

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
75,North Star Opinion Research,B/C@@8,0.043339,R+0.63@@-0.62697512,13,R+
425,Univision/Latino Decisions/North Star Opinion ...,B/C@@8,0.830409,D+1.06@@1.0564657,7,D+


In [17]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [18]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(574, 20)

In [19]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='left')
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),Voters,YouGov,Economist,11/28/2025,12/1/2025,1456.0,RV,0.844276,0.844276,...,56.0,-14.0,42.61599,54.94342,-12.327430,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/1/2025,15:00:42 2 Dec 2025,YouGov,0.047635
1,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/21/2025,12/1/2025,1500.0,LV,0.903615,0.683801,...,54.0,-9.0,42.23135,55.59648,-13.365130,https://www.rasmussenreports.com/public_conten...,12/1/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015
2,Donald Trump (2nd term),Voters,Morning Consult,NaN,11/26/2025,11/30/2025,2200.0,RV,0.711014,0.622070,...,52.0,-7.0,44.28597,52.23497,-7.949001,https://pro.morningconsult.com/trackers/donald...,11/30/2025,15:00:42 2 Dec 2025,Morning Consult,0.584994
3,Donald Trump (2nd term),Voters,The Bullfinch Group,NaN,11/21/2025,11/25/2025,1000.0,RV,0.916470,0.551256,...,54.0,-14.0,40.61126,53.73771,-13.126450,https://www.thebullfinchgroup.com/post/public-...,11/25/2025,15:00:42 2 Dec 2025,NaN,NaN
4,Donald Trump (2nd term),Voters,InsiderAdvantage,NaN,11/20/2025,11/21/2025,800.0,LV,1.093180,0.533080,...,49.0,-5.0,41.02687,51.83675,-10.809880,https://insideradvantage.com/insideradvantage-...,11/21/2025,15:00:42 2 Dec 2025,InsiderAdvantage,0.034562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,Donald Trump (2nd term),Voters,Morning Consult,NaN,1/24/2025,1/26/2025,2302.0,RV,0.795909,0.000000,...,44.0,8.0,51.28597,44.23497,7.050999,https://pro.morningconsult.com/trackers/donald...,1/26/2025,15:00:42 2 Dec 2025,Morning Consult,0.584994
570,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.0,49.23135,44.59648,4.634869,https://www.rasmussenreports.com/public_conten...,1/24/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015
571,Donald Trump (2nd term),Voters,Echelon Insights,NaN,1/22/2025,1/24/2025,1024.0,LV,1.211361,0.000000,...,43.0,8.0,50.56277,42.78031,7.782459,https://echeloninsights.com/in-the-news/dec-20...,1/24/2025,15:00:42 2 Dec 2025,Echelon Insights,-0.216672
572,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.0,53.48697,41.60001,11.886960,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,15:00:42 2 Dec 2025,RMG Research,0.278023


In [20]:
approval.shape

(574, 22)

In [21]:
approval['Pollster'].value_counts()

Pollster
Rasmussen Reports                                     219
YouGov                                                 55
Morning Consult                                        46
RMG Research                                           43
Global Strategy Group/GBAO (Navigator Research)        17
J.L. Partners                                          16
Echelon Insights                                       13
Harris Insights & Analytics                            11
Quinnipiac University                                  10
American Research Group                                10
Emerson College                                        10
John Zogby Strategies                                   8
Clarity Campaign Labs                                   7
Ipsos                                                   7
SSRS                                                    6
McLaughlin & Associates                                 6
Marquette University Law School                         6
Beaco

In [22]:
# approval[~approval['tracking'].isna()]

In [23]:
approval['tracking'].value_counts()

tracking
True    219
Name: count, dtype: int64

In [24]:
# full_approval = approval.copy()

In [25]:
approval = approval[(~approval['samplesize'].isna())]
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),Voters,YouGov,Economist,11/28/2025,12/1/2025,1456.0,RV,0.844276,0.844276,...,56.0,-14.0,42.61599,54.94342,-12.327430,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/1/2025,15:00:42 2 Dec 2025,YouGov,0.047635
1,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/21/2025,12/1/2025,1500.0,LV,0.903615,0.683801,...,54.0,-9.0,42.23135,55.59648,-13.365130,https://www.rasmussenreports.com/public_conten...,12/1/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015
2,Donald Trump (2nd term),Voters,Morning Consult,NaN,11/26/2025,11/30/2025,2200.0,RV,0.711014,0.622070,...,52.0,-7.0,44.28597,52.23497,-7.949001,https://pro.morningconsult.com/trackers/donald...,11/30/2025,15:00:42 2 Dec 2025,Morning Consult,0.584994
3,Donald Trump (2nd term),Voters,The Bullfinch Group,NaN,11/21/2025,11/25/2025,1000.0,RV,0.916470,0.551256,...,54.0,-14.0,40.61126,53.73771,-13.126450,https://www.thebullfinchgroup.com/post/public-...,11/25/2025,15:00:42 2 Dec 2025,NaN,NaN
4,Donald Trump (2nd term),Voters,InsiderAdvantage,NaN,11/20/2025,11/21/2025,800.0,LV,1.093180,0.533080,...,49.0,-5.0,41.02687,51.83675,-10.809880,https://insideradvantage.com/insideradvantage-...,11/21/2025,15:00:42 2 Dec 2025,InsiderAdvantage,0.034562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,Donald Trump (2nd term),Voters,Morning Consult,NaN,1/24/2025,1/26/2025,2302.0,RV,0.795909,0.000000,...,44.0,8.0,51.28597,44.23497,7.050999,https://pro.morningconsult.com/trackers/donald...,1/26/2025,15:00:42 2 Dec 2025,Morning Consult,0.584994
570,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.0,49.23135,44.59648,4.634869,https://www.rasmussenreports.com/public_conten...,1/24/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015
571,Donald Trump (2nd term),Voters,Echelon Insights,NaN,1/22/2025,1/24/2025,1024.0,LV,1.211361,0.000000,...,43.0,8.0,50.56277,42.78031,7.782459,https://echeloninsights.com/in-the-news/dec-20...,1/24/2025,15:00:42 2 Dec 2025,Echelon Insights,-0.216672
572,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.0,53.48697,41.60001,11.886960,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,15:00:42 2 Dec 2025,RMG Research,0.278023


In [26]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [27]:
# house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+\d.\d\d.*@@([\d\.\+\-]+)')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,-1.999072
1,D,1.017374
2,D,0.632896
3,D,0.791334
4,R,-0.597041
...,...,...
93,R,-2.443137
94,R,-0.446172
95,R,-3.234717
96,R,-4.939213


In [28]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[1]
house_eff

,0,1,house_effect
0,-1,-1.999072,-1.999072
1,1,1.017374,1.017374
2,1,0.632896,0.632896
3,1,0.791334,0.791334
4,-1,-0.597041,-0.597041
...,...,...,...
93,-1,-2.443137,-2.443137
94,-1,-0.446172,-0.446172
95,-1,-3.234717,-3.234717
96,-1,-4.939213,-4.939213


In [29]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-1.999072
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.017374
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.632896
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.791334
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.597041
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.443137
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.446172
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.234717


In [30]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['approve_adj'] = df['approve'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['disapprove_adj'] = df['disapprove'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['approve_adj'] = df['approve']
        df['disapprove_adj'] = df['disapprove']
    return df

In [31]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj
1,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/21/2025,12/1/2025,1500.0,LV,0.903615,0.683801,...,42.23135,55.59648,-13.365130,https://www.rasmussenreports.com/public_conten...,12/1/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,44.361664,54.638336
19,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/14/2025,11/20/2025,1500.0,LV,0.335273,0.090533,...,44.23135,52.59648,-8.365131,https://www.rasmussenreports.com/public_conten...,11/20/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336
22,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/20/2025,11/26/2025,1500.0,LV,0.121940,0.069691,...,43.23135,54.59648,-11.365130,https://www.rasmussenreports.com/public_conten...,11/26/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,45.361664,53.638336
24,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/19/2025,11/25/2025,1500.0,LV,0.119762,0.060984,...,44.23135,52.59648,-8.365131,https://www.rasmussenreports.com/public_conten...,11/25/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336
26,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/18/2025,11/24/2025,1500.0,LV,0.118134,0.052675,...,43.23135,52.59648,-9.365131,https://www.rasmussenreports.com/public_conten...,11/24/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,45.361664,51.638336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,Donald Trump (2nd term),Voters,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,5/19/2025,800.0,RV,1.129986,0.000000,...,46.16907,49.43924,-3.270168,https://pinpointpolicyinstitute.org/wp-content...,5/19/2025,15:00:42 2 Dec 2025,NaN,NaN,46.000000,49.000000
269,Donald Trump (2nd term),Voters,AP-NORC,NaN,5/1/2025,5/5/2025,1175.0,LV,1.145508,0.000000,...,42.16409,55.37639,-13.212300,https://apnorc.org/wp-content/uploads/2025/05/...,5/5/2025,15:00:42 2 Dec 2025,NaN,NaN,41.000000,57.000000
222,Donald Trump (2nd term),Voters,American Pulse Research & Polling,NaN,6/23/2025,6/25/2025,633.0,RV,0.813438,0.000000,...,48.61031,49.61205,-1.001740,https://s3.amazonaws.com/kajabi-storefronts-pr...,6/25/2025,15:00:42 2 Dec 2025,American Pulse Research & Polling,0.800463,49.700000,48.900000
86,Donald Trump (2nd term),Voters,"Normington, Petts & Associates",Third Way,9/11/2025,9/16/2025,800.0,LV,0.846329,0.000000,...,45.02417,53.53409,-8.509918,https://www.thirdway.org/memo/americans-caught...,9/16/2025,15:00:42 2 Dec 2025,"Normington, Petts & Associates",0.837728,45.000000,54.000000


In [32]:
approval.shape

(573, 22)

In [33]:
approval = approval_adj.copy()
approval.shape

(573, 24)

In [34]:
approval['poll_id'] = pd.factorize(approval['pollster'] + approval['enddate'] + approval['sponsors'].fillna('none'))[0]
approval.head(7)

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj,poll_id
1,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/21/2025,12/1/2025,1500.0,LV,0.903615,0.683801,...,55.59648,-13.365130,https://www.rasmussenreports.com/public_conten...,12/1/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,44.361664,54.638336,0
19,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/14/2025,11/20/2025,1500.0,LV,0.335273,0.090533,...,52.59648,-8.365131,https://www.rasmussenreports.com/public_conten...,11/20/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336,1
22,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/20/2025,11/26/2025,1500.0,LV,0.121940,0.069691,...,54.59648,-11.365130,https://www.rasmussenreports.com/public_conten...,11/26/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,45.361664,53.638336,2
24,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/19/2025,11/25/2025,1500.0,LV,0.119762,0.060984,...,52.59648,-8.365131,https://www.rasmussenreports.com/public_conten...,11/25/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336,3
26,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/18/2025,11/24/2025,1500.0,LV,0.118134,0.052675,...,52.59648,-9.365131,https://www.rasmussenreports.com/public_conten...,11/24/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,45.361664,51.638336,4
27,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/17/2025,11/21/2025,1500.0,LV,0.160467,0.051338,...,52.59648,-8.365131,https://www.rasmussenreports.com/public_conten...,11/21/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336,5
31,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/7/2025,11/13/2025,1500.0,LV,0.322031,0.033021,...,55.59648,-14.365130,https://www.rasmussenreports.com/public_conten...,11/13/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,43.361664,54.638336,6


In [35]:
approval['pollster_id'] = pd.factorize(approval['pollster'] + approval['sponsors'].fillna('none'))[0]

In [36]:
approval['population'] = approval['population'].str.strip(' ')

In [37]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id', keep=False)]
    ndup = data[~data.duplicated(subset='poll_id', keep=False)]
    # print('dup', dup, '\n ndup', ndup)
    dup_a = dup[dup['population'].isin(['RV'])]
    # print('dup_a', dup_a)
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id', keep=False)]
    ndup = df[~df.duplicated(subset='poll_id', keep=False)]
    dup_lv = dup[dup['population'].isin(['LV'])]
    # print('dup_rv', dup_lv, 'ndup', ndup)
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [38]:
approval = approval.rename({'enddate':'end_date', 'samplesize':'sample_size', 'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [39]:
approval['population'].value_counts().index.values

array(['RV', 'LV', 'V'], dtype=object)

In [40]:
approval.shape, pipeline(approval).shape

((573, 26), (571, 26))

In [41]:
pipeline(approval)

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
1,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/21/2025,2025-12-01,1500.0,LV,0.903615,0.683801,...,-13.365130,https://www.rasmussenreports.com/public_conten...,12/1/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,44.361664,54.638336,0,0
19,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/14/2025,2025-11-20,1500.0,LV,0.335273,0.090533,...,-8.365131,https://www.rasmussenreports.com/public_conten...,11/20/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336,1,0
22,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/20/2025,2025-11-26,1500.0,LV,0.121940,0.069691,...,-11.365130,https://www.rasmussenreports.com/public_conten...,11/26/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,45.361664,53.638336,2,0
24,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/19/2025,2025-11-25,1500.0,LV,0.119762,0.060984,...,-8.365131,https://www.rasmussenreports.com/public_conten...,11/25/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336,3,0
26,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/18/2025,2025-11-24,1500.0,LV,0.118134,0.052675,...,-9.365131,https://www.rasmussenreports.com/public_conten...,11/24/2025,15:00:42 2 Dec 2025,Rasmussen Reports,0.087015,45.361664,51.638336,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,Donald Trump (2nd term),Voters,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,2025-05-19,800.0,RV,1.129986,0.000000,...,-3.270168,https://pinpointpolicyinstitute.org/wp-content...,5/19/2025,15:00:42 2 Dec 2025,NaN,NaN,46.000000,49.000000,567,64
269,Donald Trump (2nd term),Voters,AP-NORC,NaN,5/1/2025,2025-05-05,1175.0,LV,1.145508,0.000000,...,-13.212300,https://apnorc.org/wp-content/uploads/2025/05/...,5/5/2025,15:00:42 2 Dec 2025,NaN,NaN,41.000000,57.000000,568,65
222,Donald Trump (2nd term),Voters,American Pulse Research & Polling,NaN,6/23/2025,2025-06-25,633.0,RV,0.813438,0.000000,...,-1.001740,https://s3.amazonaws.com/kajabi-storefronts-pr...,6/25/2025,15:00:42 2 Dec 2025,American Pulse Research & Polling,0.800463,49.700000,48.900000,569,66
86,Donald Trump (2nd term),Voters,"Normington, Petts & Associates",Third Way,9/11/2025,2025-09-16,800.0,LV,0.846329,0.000000,...,-8.509918,https://www.thirdway.org/memo/americans-caught...,9/16/2025,15:00:42 2 Dec 2025,"Normington, Petts & Associates",0.837728,45.000000,54.000000,570,67


In [42]:
# approval[approval['pollster']=='Harris Insights & Analytics']

In [43]:
full_approval = approval.copy()

In [44]:
approval = approval[approval['tracking'].isna()]

In [45]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'TIPP Insights', 'Trafalgar Group',
       'Trafalgar Group/Insider Advantage'], dtype=object)

In [46]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
0,Donald Trump (2nd term),Voters,YouGov,Economist,11/28/2025,12/1/2025,1456.0,RV,0.844276,0.844276,...,-12.327430,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/1/2025,15:00:42 2 Dec 2025,YouGov,0.047635,42.249587,55.750413,218,3
7,Donald Trump (2nd term),Voters,YouGov,Economist,11/21/2025,11/24/2025,1511.0,RV,0.690606,0.397491,...,-11.327430,https://d3nkl3psvxxpe9.cloudfront.net/document...,11/24/2025,15:00:42 2 Dec 2025,YouGov,0.047635,43.249587,55.750413,219,3
10,Donald Trump (2nd term),Voters,YouGov,Yahoo News,11/20/2025,11/24/2025,1132.0,RV,0.609293,0.312481,...,-8.327431,https://www.scribd.com/document/955164028/Yaho...,11/24/2025,15:00:42 2 Dec 2025,YouGov,0.047635,44.249587,53.750413,220,4
15,Donald Trump (2nd term),Voters,YouGov,Economist,11/15/2025,11/17/2025,1382.0,RV,0.674919,0.162654,...,-11.327430,https://d3nkl3psvxxpe9.cloudfront.net/document...,11/17/2025,15:00:42 2 Dec 2025,YouGov,0.047635,43.249587,55.750413,221,3
25,Donald Trump (2nd term),Voters,YouGov,Economist,11/7/2025,11/10/2025,1500.0,RV,0.725013,0.053300,...,-10.327430,https://d3nkl3psvxxpe9.cloudfront.net/document...,11/10/2025,15:00:42 2 Dec 2025,YouGov,0.047635,43.249587,54.750413,222,3


In [47]:
approval[approval['pollster']=='Civiqs']# [['approve', 'disapprove', 'approve_adj', 'disapprove_adj']]

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
288,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,5/17/2025,5/20/2025,1018.0,RV,1.120966,0.0,...,-4.27293,https://civiqs.com/documents/Civiqs_DailyKos_b...,5/20/2025,15:00:42 2 Dec 2025,Civiqs,0.158562,47.860928,51.139072,528,42
362,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,4/12/2025,4/15/2025,1124.0,RV,1.149724,0.0,...,-6.27293,https://civiqs.com/reports/2025/04/17/report-a...,4/15/2025,15:00:42 2 Dec 2025,Civiqs,0.158562,46.860928,52.139072,529,42
397,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,2/28/2025,3/3/2025,1031.0,RV,1.124632,0.0,...,-3.27293,https://civiqs.com/reports/2025/03/04/report-a...,3/3/2025,15:00:42 2 Dec 2025,Civiqs,0.158562,48.860928,51.139072,530,42


In [48]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'end_date', 'sample_size', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp', 'Pollster', 'pred_plus_minus', 'approve_adj',
       'disapprove_adj', 'poll_id', 'pollster_id'], dtype=object)

In [49]:
approval_polls = approval[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'approve_adj', 'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id', 'pollster_id']]
approval_polls

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,pollster_id
0,YouGov,Economist,11/28/2025,12/1/2025,1456.0,RV,42.0,56.0,42.249587,55.750413,-14.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,218,3
7,YouGov,Economist,11/21/2025,11/24/2025,1511.0,RV,43.0,56.0,43.249587,55.750413,-13.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,219,3
10,YouGov,Yahoo News,11/20/2025,11/24/2025,1132.0,RV,44.0,54.0,44.249587,53.750413,-10.000000,https://www.scribd.com/document/955164028/Yaho...,0.047635,220,4
15,YouGov,Economist,11/15/2025,11/17/2025,1382.0,RV,43.0,56.0,43.249587,55.750413,-13.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,221,3
25,YouGov,Economist,11/7/2025,11/10/2025,1500.0,RV,43.0,55.0,43.249587,54.750413,-12.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,222,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,5/19/2025,800.0,RV,46.0,49.0,46.000000,49.000000,-3.000000,https://pinpointpolicyinstitute.org/wp-content...,NaN,567,64
269,AP-NORC,NaN,5/1/2025,5/5/2025,1175.0,LV,41.0,57.0,41.000000,57.000000,-16.000000,https://apnorc.org/wp-content/uploads/2025/05/...,NaN,568,65
222,American Pulse Research & Polling,NaN,6/23/2025,6/25/2025,633.0,RV,49.7,48.9,49.700000,48.900000,0.799999,https://s3.amazonaws.com/kajabi-storefronts-pr...,0.800463,569,66
86,"Normington, Petts & Associates",Third Way,9/11/2025,9/16/2025,800.0,LV,45.0,54.0,45.000000,54.000000,-9.000000,https://www.thirdway.org/memo/americans-caught...,0.837728,570,67


In [50]:
approval_polls.shape

(343, 15)

In [51]:
def zone_flood_checker(data, pid, date):
    """
    Check whether there are multiple polls from the same pollster + same sponsor (if applicable) within the last two weeks, 
    which could indicate "flooding the zone." Return number of polls from same pollster/sponsor in last two weeks.

    :param data: Polls dataframe.
    :param pid: Pollster ID, specified in the polls dataframe as the column pollster_id.
    :param date: Date at which specified poll is conducted. Corresponds to column end_date in polls dataframe.
    """
    thres = date - pd.Timedelta(weeks=2)
    df = data[ (data['pollster_id'] == pid) & (pd.to_datetime(data['end_date']) >= thres) ]
    return max(df.shape[0], 1)

In [52]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    exp_weights = 0.1**(delta/(30))
    state_pivot['time_weights'] =  0 * linear_weights + 1 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if np.isnan(rel_qual): # For pollsters without a quality rating from the Silver Bulletin
            return 0.2
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'RV':
            return 0.8
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

   # Multiple polls in short window (aka "flooding the zone")
    def zone_flood_weight(pid, date):
        num_polls = zone_flood_checker(approval_polls, pid, date)
        return 1 / np.sqrt(num_polls)
    state_pivot['zone_flood_weight'] = state_pivot[['pollster_id', 'end_date']].apply(
        lambda x: zone_flood_weight(x['pollster_id'], x['end_date']),
        axis=1
    )

    # Gather the weights together
    state_pivot['total_weights'] = (state_pivot['sample_size_weights'] * state_pivot['time_weights'] * 
                                    state_pivot['quality_weights'] * state_pivot['population_weights'] * state_pivot['zone_flood_weight'])
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [53]:
approval_polls['sample_size'].isna().any()

False

In [54]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,...,url,pred_plus_minus,poll_id,pollster_id,sample_size_weights,time_weights,quality_weights,population_weights,zone_flood_weight,total_weights
0,YouGov,Economist,11/28/2025,2025-12-01,1456.0,RV,42.0,56.0,42.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,218,3,0.002851,9.261187e-01,0.964021,0.8,0.577350,9.019267e-02
7,YouGov,Economist,11/21/2025,2025-11-24,1511.0,RV,43.0,56.0,43.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,219,3,0.002904,5.411695e-01,0.964021,0.8,0.500000,4.649648e-02
10,YouGov,Yahoo News,11/20/2025,2025-11-24,1132.0,RV,44.0,54.0,44.249587,53.750413,...,https://www.scribd.com/document/955164028/Yaho...,0.047635,220,4,0.002514,5.411695e-01,0.964021,0.8,1.000000,8.048984e-02
15,YouGov,Economist,11/15/2025,2025-11-17,1382.0,RV,43.0,56.0,43.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,221,3,0.002777,3.162278e-01,0.964021,0.8,0.447214,2.324093e-02
25,YouGov,Economist,11/7/2025,2025-11-10,1500.0,RV,43.0,55.0,43.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,222,3,0.002893,1.847850e-01,0.964021,0.8,0.408248,1.291580e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,2025-05-19,800.0,RV,46.0,49.0,46.000000,49.000000,...,https://pinpointpolicyinstitute.org/wp-content...,NaN,567,64,0.002113,2.712273e-07,0.200000,0.8,1.000000,7.035685e-09
269,AP-NORC,NaN,5/1/2025,2025-05-05,1175.0,LV,41.0,57.0,41.000000,57.000000,...,https://apnorc.org/wp-content/uploads/2025/05/...,NaN,568,65,0.002561,9.261187e-08,0.200000,1.0,1.000000,3.639349e-09
222,American Pulse Research & Polling,NaN,6/23/2025,2025-06-25,633.0,RV,49.7,48.9,49.700000,48.900000,...,https://s3.amazonaws.com/kajabi-storefronts-pr...,0.800463,569,66,0.001880,4.641589e-06,0.200000,0.8,1.000000,1.071018e-07
86,"Normington, Petts & Associates",Third Way,9/11/2025,2025-09-16,800.0,LV,45.0,54.0,45.000000,54.000000,...,https://www.thirdway.org/memo/americans-caught...,0.837728,570,67,0.002113,2.712273e-03,0.200000,1.0,1.000000,8.794607e-05


In [55]:
avgs_pipeline(approval_polls, datetime.datetime.today()).shape

(343, 21)

In [56]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['approve_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['disapprove_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [57]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Approve,Disapprove
0,2025-01-23,56.196279,39.233963
1,2025-01-24,53.679015,41.058367
2,2025-01-25,53.679015,41.058367
3,2025-01-26,53.609156,41.198826
4,2025-01-27,50.853303,42.230873
...,...,...,...
309,2025-11-28,43.131357,53.867116
310,2025-11-29,43.131357,53.867116
311,2025-11-30,43.185167,53.813341
312,2025-12-01,43.100785,53.988050


In [58]:
polls_df.shape

(343, 21)

In [59]:
polls_df.head(3)

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,...,url,pred_plus_minus,poll_id,pollster_id,sample_size_weights,time_weights,quality_weights,population_weights,zone_flood_weight,total_weights
0,YouGov,Economist,11/28/2025,2025-12-01,1456.0,RV,42.0,56.0,42.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,218,3,0.002851,0.926119,0.964021,0.8,0.57735,0.090193
7,YouGov,Economist,11/21/2025,2025-11-24,1511.0,RV,43.0,56.0,43.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,219,3,0.002904,0.541170,0.964021,0.8,0.50000,0.046496
10,YouGov,Yahoo News,11/20/2025,2025-11-24,1132.0,RV,44.0,54.0,44.249587,53.750413,...,https://www.scribd.com/document/955164028/Yaho...,0.047635,220,4,0.002514,0.541170,0.964021,0.8,1.00000,0.080490


In [60]:
polls_df['net_adj'] = polls_df['approve_adj'] - polls_df['disapprove_adj']

In [61]:
display_df = polls_df[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'net',
                      'total_weights', 'url']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 'total_weights':'Weight', 'startdate':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population']
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df['Pollster'] = '[' + display_df['Pollster'] + '](' + display_df['URL'] + ')'
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net
0,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-11-28,2025-12-01,1456 RV,9.019267e-02,42.0,56.0,-14.0
1,"<a href=""https://pro.morningconsult.com/tracke...",2025-11-26,2025-11-30,2200 RV,2.130160e-02,45.0,52.0,-7.0
2,"<a href=""https://www.thebullfinchgroup.com/pos...",2025-11-21,2025-11-25,1000 RV,1.694707e-02,40.0,54.0,-14.0
3,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-11-21,2025-11-24,1511 RV,4.649648e-02,43.0,56.0,-13.0
4,"<a href=""https://www.scribd.com/document/95516...",2025-11-20,2025-11-24,1132 RV,8.048984e-02,44.0,54.0,-10.0
...,...,...,...,...,...,...,...,...
338,"<a href=""https://poll.qu.edu/poll-release?rele...",2025-01-23,2025-01-27,1019 RV,2.900868e-12,46.0,43.0,3.0
339,"<a href=""https://pro.morningconsult.com/tracke...",2025-01-24,2025-01-26,2302 RV,3.044676e-13,52.0,44.0,8.0
340,"<a href=""https://echeloninsights.com/in-the-ne...",2025-01-22,2025-01-24,1024 LV,2.490300e-12,51.0,43.0,8.0
341,"<a href=""https://x.com/ScottWRasmussen/status/...",2025-01-20,2025-01-23,3000 RV,1.140280e-12,57.0,39.0,18.0


In [62]:
display_df.to_csv('polls_display_table_RV.csv')

In [63]:
approval_std

,Date,Approve,Disapprove
0,2025-01-23,0.756733,0.261623
1,2025-01-24,2.653116,1.891457
2,2025-01-25,2.653116,1.891457
3,2025-01-26,2.593806,1.926309
4,2025-01-27,3.829991,2.043437
...,...,...,...
309,2025-11-28,2.483631,3.180068
310,2025-11-29,2.483631,3.180068
311,2025-11-30,2.478866,3.161752
312,2025-12-01,2.379577,3.066423


In [64]:
approval_avg['Net'] = approval_avg['Approve'] - approval_avg['Disapprove']
approval_avg

,Date,Approve,Disapprove,Net
0,2025-01-23,56.196279,39.233963,16.962316
1,2025-01-24,53.679015,41.058367,12.620648
2,2025-01-25,53.679015,41.058367,12.620648
3,2025-01-26,53.609156,41.198826,12.410330
4,2025-01-27,50.853303,42.230873,8.622431
...,...,...,...,...
309,2025-11-28,43.131357,53.867116,-10.735759
310,2025-11-29,43.131357,53.867116,-10.735759
311,2025-11-30,43.185167,53.813341,-10.628173
312,2025-12-01,43.100785,53.988050,-10.887265


In [65]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Approve_x'] + 1.96*approval_trendline['Approve_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Approve_x'] - 1.96*approval_trendline['Approve_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Disapprove_x'] + 1.96*approval_trendline['Disapprove_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Disapprove_x'] - 1.96*approval_trendline['Disapprove_y']
approval_trendline = approval_trendline.rename({
    'Approve_x':'approve_mean',
    'Disapprove_x':'disapprove_mean',
}, axis='columns').drop(['Approve_y', 'Disapprove_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-23,56.196279,39.233963,57.679476,54.713083,39.746744,38.721182
1,2025-01-24,53.679015,41.058367,58.879122,48.478908,44.765623,37.351110
2,2025-01-25,53.679015,41.058367,58.879122,48.478908,44.765623,37.351110
3,2025-01-26,53.609156,41.198826,58.693015,48.525297,44.974393,37.423260
4,2025-01-27,50.853303,42.230873,58.360086,43.346520,46.236008,38.225737
...,...,...,...,...,...,...,...
309,2025-11-28,43.131357,53.867116,47.999272,38.263441,60.100049,47.634182
310,2025-11-29,43.131357,53.867116,47.999272,38.263441,60.100049,47.634182
311,2025-11-30,43.185167,53.813341,48.043744,38.326590,60.010374,47.616307
312,2025-12-01,43.100785,53.988050,47.764756,38.436813,59.998240,47.977861


In [66]:
# Export approval ratings
approval_trendline.to_csv('approval_stats_RV.csv')

In [67]:
polls_df.shape

(343, 22)

In [68]:
line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
line.update_traces(hovertemplate=None)
line.update_layout(hovermode='x unified')
# scatter.update_traces(hovertemplate=None, hoverinfo='skip')
fig_approve_CI = go.Figure([
    go.Scatter(
        name='Approve CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Approve CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])
fig_disapprove_CI = go.Figure([
    go.Scatter(
        name='Disapprove CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Disapprove CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])

fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
fig.update_layout(
    title='President Trump Approval Rating',
    xaxis_title='Date',
    yaxis_title='%',
    legend_title='Legend',
    hovermode='x unified'
)
fig.show()

In [69]:
line = px.line(approval_avg, x='Date', y=['Net'])
scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net'])
                     #trendline='lowess', trendline_options=dict(frac=0.2))
fig = go.Figure(data=line.data + scatter.data)
fig.add_hline(y=0, line_dash='dash')
fig.show()

In [70]:
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net', 'net_adj'])
# fig = go.Figure(data=scatter.data)
# fig.add_hline(y=0, line_dash='dash')
# fig.show()

In [71]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['approve_adj'] - data['disapprove_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [72]:
net_approval = net_col(approval, datetime.date(2025, 1, 23))
net_approval

,Date,net,net_std
0,2025-01-23,16.962316,1.018356
1,2025-01-24,12.620648,4.538280
2,2025-01-25,12.620648,4.538280
3,2025-01-26,12.410330,4.493460
4,2025-01-27,8.622431,5.487000
...,...,...,...
309,2025-11-28,-10.735759,4.904112
310,2025-11-29,-10.735759,4.904112
311,2025-11-30,-10.628173,4.895917
312,2025-12-01,-10.887265,4.741861


In [73]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-23,16.962316,1.018356,18.958293,14.966339
1,2025-01-24,12.620648,4.538280,21.515677,3.725620
2,2025-01-25,12.620648,4.538280,21.515677,3.725620
3,2025-01-26,12.410330,4.493460,21.217511,3.603148
4,2025-01-27,8.622431,5.487000,19.376950,-2.132089
...,...,...,...,...,...
309,2025-11-28,-10.735759,4.904112,-1.123700,-20.347818
310,2025-11-29,-10.735759,4.904112,-1.123700,-20.347818
311,2025-11-30,-10.628173,4.895917,-1.032175,-20.224171
312,2025-12-01,-10.887265,4.741861,-1.593219,-20.181312


In [74]:
net_approval.to_csv('net_approval_RV.csv')

### Auxiliary Graph: Individual Pollsters

In [76]:
approval_polls.columns.values

array(['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size',
       'population', 'approve', 'disapprove', 'approve_adj',
       'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id',
       'pollster_id'], dtype=object)

In [77]:
appr = full_approval.copy()
appr['net_adj'] = appr['approve'] - appr['disapprove']
appr = appr[~appr['pred_plus_minus'].isna()]
pollsters = appr['pollster'].value_counts().index.values
pollsters

array(['Rasmussen Reports', 'YouGov', 'Morning Consult', 'RMG Research',
       'Global Strategy Group/GBAO (Navigator Research)', 'J.L. Partners',
       'Echelon Insights', 'Harris Insights & Analytics',
       'American Research Group', 'Quinnipiac University',
       'Emerson College', 'John Zogby Strategies', 'Ipsos',
       'Clarity Campaign Labs', 'Beacon Research/Shaw & Co. Research',
       'Marquette University Law School', 'InsiderAdvantage',
       'McLaughlin & Associates', 'SSRS', 'Marist College',
       'TIPP Insights', 'Cygnal', 'Wick', 'co/efficient',
       'Targoz Market Research',
       'Fabrizio Lee & Associates/Impact Research', 'Civiqs',
       'Hart Research Associates/Public Opinion Strategies', 'NewsNation',
       'GBAO', 'The New York Times/Siena College',
       'OH Predictive Insights', 'Trafalgar Group', 'Blueprint Polling',
       'The Winston Group', 'OnMessage Inc.', 'Remington Research Group',
       'North Star Opinion Research', 'Public Opinion St

In [78]:
def approval_today(data: pd.DataFrame):
    # date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
#     yes_averages = []
#     no_averages = []
#     yes_stdevs = []
#     no_stdevs = []
    pipelined_df = avgs_pipeline(data, datetime.datetime.today())
    yes_avg = np.sum(pipelined_df['approve'] * pipelined_df['total_weights'])
    no_avg = np.sum(pipelined_df['disapprove'] * pipelined_df['total_weights'])
    yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve'] - yes_avg)**2))
    no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove'] - no_avg)**2))
    net_avg = np.sum(pipelined_df['net'] * pipelined_df['total_weights'])
    net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net'] - net_avg)**2))
    
    #return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [79]:
def indiv_pollster_avg(pollster: str):
    df = appr[appr['pollster'] == pollster]
    yes_avg, no_avg, yes_std, no_std, net_avg, net_std = approval_today(df)
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [80]:
def get_ppm(pollster: str):
    df = ratings[ratings['Pollster'] == pollster]
    ppm = df['Predictive Plus-Minus'].values[0]
    
    return ppm

In [81]:
appr.shape

(551, 27)

In [82]:
def get_num_polls(pollster: str):
    df = appr[appr['pollster'] == pollster]
    num_polls = df.shape[0]
    return num_polls

In [83]:
get_ppm('YouGov')

0.047635015

In [84]:
indiv_pollster_avg('YouGov')

(42.971940510654164,
 55.281822112310095,
 0.7897891837131478,
 0.8925559128559915,
 -12.309881601655952,
 1.6273250400350439)

In [85]:
indiv_pollster_avg('RMG Research')

(47.26387616041294,
 51.284718896138116,
 1.7279598545272477,
 1.846648273939728,
 -4.020842735725186,
 3.4827794181280387)

In [86]:
nets = []
net_stdevs = []
ppms = []
nums = []

for p in pollsters:
    _, _, _, _, net, net_std = indiv_pollster_avg(p)
    ppm = get_ppm(p)
    num_polls = get_num_polls(p)
    
    nets.append(net)
    net_stdevs.append(net_std)
    ppms.append(ppm)
    nums.append(num_polls)

In [87]:
# nets

In [88]:
indiv_avgs = pd.DataFrame({'Pollster': pollsters, 'Net Approval': nets, 'Net Stdev': net_stdevs, 'Predictive Plus-Minus': ppms, 'Number of Approval Polls Conducted': nums})
indiv_avgs = indiv_avgs.sort_values(['Net Approval'], ascending=True)
indiv_avgs

,Pollster,Net Approval,Net Stdev,Predictive Plus-Minus,Number of Approval Polls Conducted
8,American Research Group,-26.723043,1.045747,0.818141,10
18,SSRS,-21.998431,0.125234,0.966835,5
14,Beacon Research/Shaw & Co. Research,-16.967644,0.538660,-0.635562,6
19,Marist College,-16.893694,0.791659,-0.436610,5
12,Ipsos,-16.610781,2.006344,0.442898,7
4,Global Strategy Group/GBAO (Navigator Research),-15.965449,2.402787,0.804988,17
6,Echelon Insights,-14.350582,2.176384,-0.216672,13
9,Quinnipiac University,-14.166086,0.600263,-0.290484,10
28,NewsNation,-13.000000,0.000656,0.614459,2
1,YouGov,-12.309882,1.627325,0.047635,55


In [89]:
# indiv_avgs.to_csv('indiv_pollster_avgs_RV.csv')

In [90]:
reg = lm.LinearRegression()
reg.fit(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
reg.coef_, reg.intercept_

(array([-2.35642405]), -7.720544592284126)

In [91]:
r = r_regression(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
r

array([-0.18633528])